# imports 

In [3]:
#standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import ticker, cm
import seaborn as sns
from scipy import signal,ndimage,linalg

from glob import glob
import os
import gc

# Data import 

In [4]:
filepaths=glob ("Mechanocardiograms_with_ECG_reference/*.txt")
print (filepaths)

['Mechanocardiograms_with_ECG_reference\\sub_1.txt', 'Mechanocardiograms_with_ECG_reference\\sub_10.txt', 'Mechanocardiograms_with_ECG_reference\\sub_11.txt', 'Mechanocardiograms_with_ECG_reference\\sub_12.txt', 'Mechanocardiograms_with_ECG_reference\\sub_13.txt', 'Mechanocardiograms_with_ECG_reference\\sub_14.txt', 'Mechanocardiograms_with_ECG_reference\\sub_15.txt', 'Mechanocardiograms_with_ECG_reference\\sub_16.txt', 'Mechanocardiograms_with_ECG_reference\\sub_17.txt', 'Mechanocardiograms_with_ECG_reference\\sub_18.txt', 'Mechanocardiograms_with_ECG_reference\\sub_19.txt', 'Mechanocardiograms_with_ECG_reference\\sub_2.txt', 'Mechanocardiograms_with_ECG_reference\\sub_20.txt', 'Mechanocardiograms_with_ECG_reference\\sub_21.txt', 'Mechanocardiograms_with_ECG_reference\\sub_22.txt', 'Mechanocardiograms_with_ECG_reference\\sub_23.txt', 'Mechanocardiograms_with_ECG_reference\\sub_24.txt', 'Mechanocardiograms_with_ECG_reference\\sub_25.txt', 'Mechanocardiograms_with_ECG_reference\\sub_26.

In [8]:
data=np.loadtxt(filepaths[0],skiprows=15)
data_df=pd.DataFrame(data,columns=["ECG","Acc_X","Acc_Y","Acc_Z","Gyro_X","Gyro_Y","Gyro_Z"])
display(data_df)

,ECG,Acc_X,Acc_Y,Acc_Z,Gyro_X,Gyro_Y,Gyro_Z
0,-436.0,6080.0,16892.0,-1704.0,0.0,0.0,0.0
1,446.0,-712.0,-1160.0,-16448.0,1024.0,1536.0,-256.0
2,-553.0,-688.0,-1056.0,-16432.0,1024.0,1792.0,0.0
3,-726.0,-736.0,-1072.0,-16408.0,2048.0,2048.0,-257.0
4,236.0,-712.0,-1112.0,-16408.0,1024.0,2048.0,-513.0
...,...,...,...,...,...,...,...
146683,819.0,-344.0,-1392.0,-16328.0,20736.0,-9729.0,9728.0
146684,509.0,-320.0,-1456.0,-16248.0,20736.0,-8705.0,9728.0
146685,-824.0,-312.0,-1400.0,-16312.0,19712.0,-7681.0,8704.0
146686,807.0,-344.0,-1416.0,-16320.0,19456.0,-7681.0,9728.0


# Data parser loop

In [18]:
for mypath in filepaths:
    head, tail = os.path.split(mypath)
    print (head,tail)
    print(tail.split(".")[0])
    key=tail.split(".")[0]
    
    start_string = '[SENSORS]'
    end_string = '[DATA]'

    # Variables to track the state and storage for lines
    start_index = None
    end_index = None
    is_collecting = False
    lines_between = []

    # Open the file in read mode
    with open(mypath, 'r') as file:
        # Initialize line index
        line_index = 0

        # Iterate over each line in the file
        for line in file:
            # Increment line index
            line_index += 1

            # Check if we're at the start string
            if start_string in line and not is_collecting:
                start_index = line_index
                is_collecting = True
                continue  # Skip adding the start line to the list

            # Check if we've found the end string
            elif end_string in line and is_collecting:
                end_index = line_index
                break  # Stop collecting after finding the end string

            # Collect lines between start and end strings
            if is_collecting:
                lines_between.append(line.strip())

    # Print results
    if start_index and end_index:
        print(f"Start string found at line {start_index}")
        print(f"End string found at line {end_index}")
        print("Lines between the substrings:", lines_between)
    else:
        print("Start and/or end string not found in file.")
   
    print("")
    mycolumns=[]
    for c in lines_between[1:]:
        mycolumns.append(c.split(",")[0])
    print(mycolumns)
    print("")
    print("")
    print("")
    
    
    data=np.loadtxt(mypath,skiprows=end_index)
    data_df=pd.DataFrame(data,columns=mycolumns)
    display(data_df)
    print (data_df.head())
    print (data_df.shape)
    print("")
    print ("saving formatted csv: ",key)
    data_df.to_csv("Formatted_MCG_With_ECG_Reference_Data/{0}".format(key),index=False)
    print("")
    print("")
    print("")
    
    
    
    

Mechanocardiograms_with_ECG_reference sub_1.txt
sub_1
Start string found at line 6
End string found at line 15
Lines between the substrings: ['Sensor count = 7', 'Signal1: EKG, F-EKG V.3 16072013', 'Signal2: accX, MMA8451Q', 'Signal3: accY, MMA8451Q', 'Signal4: accZ, MMA8451Q', 'Signal5: gyroX, Maxim MAX21000', 'Signal6: gyroY, Maxim MAX21000', 'Signal7: gyroZ, Maxim MAX21000']

['Signal1: EKG', 'Signal2: accX', 'Signal3: accY', 'Signal4: accZ', 'Signal5: gyroX', 'Signal6: gyroY', 'Signal7: gyroZ']





,Signal1: EKG,Signal2: accX,Signal3: accY,Signal4: accZ,Signal5: gyroX,Signal6: gyroY,Signal7: gyroZ
0,-436.0,6080.0,16892.0,-1704.0,0.0,0.0,0.0
1,446.0,-712.0,-1160.0,-16448.0,1024.0,1536.0,-256.0
2,-553.0,-688.0,-1056.0,-16432.0,1024.0,1792.0,0.0
3,-726.0,-736.0,-1072.0,-16408.0,2048.0,2048.0,-257.0
4,236.0,-712.0,-1112.0,-16408.0,1024.0,2048.0,-513.0
...,...,...,...,...,...,...,...
146683,819.0,-344.0,-1392.0,-16328.0,20736.0,-9729.0,9728.0
146684,509.0,-320.0,-1456.0,-16248.0,20736.0,-8705.0,9728.0
146685,-824.0,-312.0,-1400.0,-16312.0,19712.0,-7681.0,8704.0
146686,807.0,-344.0,-1416.0,-16320.0,19456.0,-7681.0,9728.0


   Signal1: EKG  Signal2: accX  Signal3: accY  Signal4: accZ  Signal5: gyroX  \
0        -436.0         6080.0        16892.0        -1704.0             0.0   
1         446.0         -712.0        -1160.0       -16448.0          1024.0   
2        -553.0         -688.0        -1056.0       -16432.0          1024.0   
3        -726.0         -736.0        -1072.0       -16408.0          2048.0   
4         236.0         -712.0        -1112.0       -16408.0          1024.0   

   Signal6: gyroY  Signal7: gyroZ  
0             0.0             0.0  
1          1536.0          -256.0  
2          1792.0             0.0  
3          2048.0          -257.0  
4          2048.0          -513.0  
(146688, 7)

saving formatted csv:  sub_1



Mechanocardiograms_with_ECG_reference sub_10.txt
sub_10
Start string found at line 7
End string found at line 19
Lines between the substrings: ['Sensor count = 10', 'Signal1: ECG, TI ADS1293', 'Signal2: accX, MMA8451Q', 'Signal3: accY, MMA8451Q', 'Signal4: a

,Signal1: ECG,Signal2: accX,Signal3: accY,Signal4: accZ,Signal5: gyroX,Signal6: gyroY,Signal7: gyroZ,Signal8: ECG1,Signal9: ECG2,Signal10: NC
0,0.0,10524.0,-13332.0,-8960.0,0.0,0.0,0.0,7581.0,7581.0,0.0
1,-3543.0,2480.0,-2128.0,-15752.0,98.0,23.0,-82.0,31878.0,28335.0,0.0
2,-3579.0,2488.0,-2144.0,-15800.0,89.0,-24.0,-107.0,31880.0,28301.0,0.0
3,-3588.0,2512.0,-2112.0,-15712.0,37.0,-34.0,-77.0,31930.0,28343.0,0.0
4,-3582.0,2504.0,-2160.0,-15752.0,33.0,-63.0,-99.0,31960.0,28379.0,0.0
...,...,...,...,...,...,...,...,...,...,...
492091,1006.0,2472.0,-2536.0,-16008.0,-5.0,-106.0,-151.0,32032.0,-32498.0,0.0
492092,1101.0,2424.0,-2560.0,-16024.0,-5.0,-116.0,-160.0,32050.0,-32385.0,0.0
492093,1098.0,2344.0,-2480.0,-16024.0,40.0,-112.0,-156.0,32039.0,-32398.0,0.0
492094,1051.0,2344.0,-2568.0,-16032.0,43.0,-127.0,-164.0,32080.0,-32405.0,0.0


   Signal1: ECG  Signal2: accX  Signal3: accY  Signal4: accZ  Signal5: gyroX  \
0           0.0        10524.0       -13332.0        -8960.0             0.0   
1       -3543.0         2480.0        -2128.0       -15752.0            98.0   
2       -3579.0         2488.0        -2144.0       -15800.0            89.0   
3       -3588.0         2512.0        -2112.0       -15712.0            37.0   
4       -3582.0         2504.0        -2160.0       -15752.0            33.0   

   Signal6: gyroY  Signal7: gyroZ  Signal8: ECG1  Signal9: ECG2  Signal10: NC  
0             0.0             0.0         7581.0         7581.0           0.0  
1            23.0           -82.0        31878.0        28335.0           0.0  
2           -24.0          -107.0        31880.0        28301.0           0.0  
3           -34.0           -77.0        31930.0        28343.0           0.0  
4           -63.0           -99.0        31960.0        28379.0           0.0  
(492096, 10)

saving formatted csv:  su

...

   Signal1: EKG  Signal2: accX  Signal3: accY  Signal4: accZ  Signal5: gyroX  \
0       -4841.0         6080.0        16892.0        -1704.0             0.0   
1       -1148.0          184.0          632.0       -16360.0         -1025.0   
2        -841.0          128.0          560.0       -16472.0          -513.0   
3       -1008.0          136.0          600.0       -16456.0         -1025.0   
4       -1180.0          192.0          568.0       -16472.0         -2305.0   

   Signal6: gyroY  Signal7: gyroZ  
0             0.0             0.0  
1             0.0           512.0  
2          -257.0             0.0  
3          -257.0          -256.0  
4         -1025.0           512.0  
(145920, 7)

saving formatted csv:  sub_7



Mechanocardiograms_with_ECG_reference sub_8.txt
sub_8
Start string found at line 7
End string found at line 17
Lines between the substrings: ['Sensor count = 8', 'Signal1: ECG, TI ADS1293', 'Signal2: accX, MMA8451Q', 'Signal3: accY, MMA8451Q', 'Signal4: accZ

,Signal1: ECG,Signal2: accX,Signal3: accY,Signal4: accZ,Signal5: gyroX,Signal6: gyroY,Signal7: gyroZ,Signal8: NC
0,7574.0,-32516.0,-15912.0,-11904.0,0.0,0.0,0.0,0.0
1,4721.0,-2624.0,4000.0,-15608.0,-35.0,-59.0,27.0,0.0
2,4725.0,-2568.0,4008.0,-15544.0,-43.0,-69.0,37.0,0.0
3,4695.0,-2576.0,4024.0,-15584.0,-32.0,-79.0,22.0,0.0
4,4670.0,-2576.0,4064.0,-15584.0,-10.0,-50.0,13.0,0.0
...,...,...,...,...,...,...,...,...
244795,7306.0,-4032.0,3776.0,-15304.0,14.0,124.0,32.0,0.0
244796,7362.0,-4080.0,3800.0,-15344.0,30.0,136.0,17.0,0.0
244797,7278.0,-3984.0,3848.0,-15352.0,12.0,139.0,20.0,0.0
244798,7367.0,-4000.0,3800.0,-15256.0,35.0,109.0,8.0,0.0


   Signal1: ECG  Signal2: accX  Signal3: accY  Signal4: accZ  Signal5: gyroX  \
0        7574.0       -32516.0       -15912.0       -11904.0             0.0   
1        4721.0        -2624.0         4000.0       -15608.0           -35.0   
2        4725.0        -2568.0         4008.0       -15544.0           -43.0   
3        4695.0        -2576.0         4024.0       -15584.0           -32.0   
4        4670.0        -2576.0         4064.0       -15584.0           -10.0   

   Signal6: gyroY  Signal7: gyroZ  Signal8: NC  
0             0.0             0.0          0.0  
1           -59.0            27.0          0.0  
2           -69.0            37.0          0.0  
3           -79.0            22.0          0.0  
4           -50.0            13.0          0.0  
(244800, 8)

saving formatted csv:  sub_8



Mechanocardiograms_with_ECG_reference sub_9.txt
sub_9
Start string found at line 7
End string found at line 17
Lines between the substrings: ['Sensor count = 8', 'Signal1: ECG, TI 

,Signal1: ECG,Signal2: accX,Signal3: accY,Signal4: accZ,Signal5: gyroX,Signal6: gyroY,Signal7: gyroZ,Signal8: ECG1
0,7575.0,-32260.0,-15912.0,-11904.0,0.0,0.0,0.0,0.0
1,10732.0,2544.0,-520.0,-16416.0,-360.0,818.0,-108.0,-21484.0
2,10636.0,2448.0,-520.0,-16488.0,-319.0,844.0,-107.0,-21450.0
3,10632.0,2576.0,-552.0,-16424.0,-277.0,802.0,-110.0,-21659.0
4,10882.0,2592.0,-584.0,-16496.0,-240.0,816.0,-80.0,-21987.0
...,...,...,...,...,...,...,...,...
483355,17476.0,-320.0,-1296.0,-16168.0,80.0,-761.0,98.0,-22205.0
483356,17496.0,-376.0,-1240.0,-16080.0,65.0,-769.0,50.0,-22138.0
483357,17539.0,-344.0,-1264.0,-16176.0,47.0,-729.0,-3.0,-21993.0
483358,17596.0,-320.0,-1248.0,-16128.0,61.0,-719.0,-52.0,-22072.0


   Signal1: ECG  Signal2: accX  Signal3: accY  Signal4: accZ  Signal5: gyroX  \
0        7575.0       -32260.0       -15912.0       -11904.0             0.0   
1       10732.0         2544.0         -520.0       -16416.0          -360.0   
2       10636.0         2448.0         -520.0       -16488.0          -319.0   
3       10632.0         2576.0         -552.0       -16424.0          -277.0   
4       10882.0         2592.0         -584.0       -16496.0          -240.0   

   Signal6: gyroY  Signal7: gyroZ  Signal8: ECG1  
0             0.0             0.0            0.0  
1           818.0          -108.0       -21484.0  
2           844.0          -107.0       -21450.0  
3           802.0          -110.0       -21659.0  
4           816.0           -80.0       -21987.0  
(483360, 8)

saving formatted csv:  sub_9



